In [1]:
import numpy as np
import timeit
import scipy.linalg
import warnings
import math
import matplotlib.pyplot as plt
import pandas as pd
import random
import cmath
from scipy.interpolate import CubicSpline

warnings.filterwarnings('ignore')

# Dzień dobry

na starcie wkleją implementację metod, których używałem już na wcześniejszych listach - lub pochodzą z prezentacji prof. Szwabińskiego.

## Sim

In [2]:
# Source: Kody do wykładu 8
def simps(f,a,b,N=50):
    #funkcja f musi być zwektoryzowana
    if N % 2 == 1:
        raise ValueError("N must be an even integer.")
    dx = (b-a)/N
    x = np.linspace(a,b,N+1)
    y = f(x)
    S = dx/3 * np.sum(y[0:-1:2] + 4*y[1::2] + y[2::2])
    return S

## Kwadratura Gaussa-Legendre'a

In [3]:
# Source: Kody do wykładu 8
def gaussNodes(m,tol=10e-9):
    def legendre(t,m):
        p0 = 1.0; p1 = t
        for k in range(1,m):
            p = ((2.0*k + 1.0)*t*p1 - k*p0)/(1.0 + k )
            p0 = p1; p1 = p
        dp = m*(p0 - t*p1)/(1.0 - t**2)
        return p,dp
    
    A = np.zeros(m)
    x = np.zeros(m)
    nRoots = int((m + 1)/2) # liczba nieujemnych pierwiastków

    for i in range(nRoots):
        t = math.cos(math.pi*(i + 0.75)/(m + 0.5))# przybliżony pierwiastek

        for j in range(30):
            p,dp = legendre(t,m)# metoda Newtona-Raphsona
            dt = -p/dp; t = t + dt# 
            if abs(dt) < tol:
                x[i] = t; x[m-i-1] = -t
                A[i] = 2.0/(1.0 - t**2)/(dp**2) # Eq.(6.25)
                A[m-i-1] = A[i]
                break
    return x,A

In [4]:
def gaussQuad(f,a,b,m):
    c1 = (b + a)/2.0
    c2 = (b - a)/2.0
    x,A = gaussNodes(m)
    sum = 0.0
    for i in range(len(x)):
        sum = sum + A[i]*f(c1 + c2*x[i])
    return c2*sum

# zad 1

In [5]:
v = np.array([1.0, 1.8, 2.4, 3.5, 4.4, 5.1, 6.0])  # Prędkości (m/s)
P = np.array([4.7, 12.2, 19.0, 31.8, 40.1, 43.8, 43.2])  # Moc (kW)

# Konwersja jednostek
P = P * 1000  # kW -> W
m = 2000  # Masa w kg

# Funkcja podcałkowa
def integrand(v_query):
    P_query = np.interp(v_query, v, P)
    return v_query / P_query

# Przedział całkowania
v1, v2 = 1.0, 6.0

# Liczba węzłów (parzysta)
N = 50

# Obliczenie czasu przyspieszenia
delta_t = m * simps(integrand, v1, v2, N)
print(f"Czas przyspieszenia: {delta_t:.2f} s")

Czas przyspieszenia: 1.28 s


# zad 2

In [6]:
# Funkcja podcałkowa
def f(x):
    return np.cos(2 * np.arccos(x))


a, b = -1, 1

nodes = [3, 5, 7]

for n in nodes:
    N = n - 1
    result = simps(f, a, b, N)
    print(f"Całka dla {n} węzłów: {result:.6f}")


Całka dla 3 węzłów: -0.666667
Całka dla 5 węzłów: -0.666667
Całka dla 7 węzłów: -0.666667


Zauważmy, że

$$
f(x) = \cos\big(2 \cos^{-1}(x)\big).
$$

Z tożsamości trygonometrycznej:
$$
\cos(2\theta) = 2\cos^2(\theta) - 1,
$$
gdzie $\theta = \cos^{-1}(x)$, wynika, że:
$$
\cos\big(2 \cos^{-1}(x)\big) = 2\cos^2\big(\cos^{-1}(x)\big) - 1.
$$

Dla $\cos(\cos^{-1}(x)) = x$, otrzymujemy:
$$
f(x) = 2x^2 - 1.
$$

W ten sposób funkcja podcałkowa została sprowadzona do wielomianu kwadratowego:
$$
f(x) = 2x^2 - 1.
$$

Wzór Simpsona działa dokładnie dla wielomianów stopnia co najwyżej trzeciego. Wynika to z faktu, że wzór Simpsona jest formą kwadratury numerycznej opartą na przybliżeniu funkcji wielomianem kwadratowym na każdym podprzedziale.

Ponieważ funkcja f(x)=2x2−1f(x)=2x2−1 jest wielomianem stopnia 2, całka numeryczna obliczona przy użyciu wzoru Simpsona będzie dokładna niezależnie od liczby węzłów (o ile NN jest parzyste i wystarczające dla dokładności arytmetycznej).

# zad 3

Do policzenia całki z zadania zastosujemy dane podstawienie:
$$
x = \frac{1}{t^{\frac{1}{3}}}
$$
$$
dx = -\frac{1}{3}\frac{1}{t^{\frac{4}{3}}}dt
$$
Należy pamiętać o zamianie granic całkowania
$$
\int_1^{\infty} \frac{1}{1+x^4}dx = \int_1^0 \frac{1}{1+(\frac{1}{t^{\frac{1}{3}}})^4}(-\frac{1}{3}\frac{1}{t^{\frac{4}{3}}})dt = -\frac{1}{3}\int_1^0\frac{1}{1+t^{\frac{4}{3}}} dt = \frac{1}{3}\int_0^1\frac{1}{1+t^{\frac{4}{3}}} dt
$$
Tą funkcję podcałkową możemy teraz zastosować w obliczeniach

In [ ]:
def integrand(t):
    return (1/3) * (1/(1 + t**(4/3)))

# Przybliża funkcję podcałkową za pomocą trapezów
def trapezoidal_rule(f, a, b, n):
    x = np.linspace(a, b, n)
    y = f(x)
    h = (b - a) / (n - 1)
    T = h * (0.5 * y[0] + np.sum(y[1:-1]) + 0.5 * y[-1])
    return T

# Obliczenia
a, b = 0, 1
n = 6
result = trapezoidal_rule(integrand, a, b, n)
print(f"Całka: {result:.6f}")


Całka: 0.243698


# zad 4

In [15]:
g = 9.81  # Przyspieszenie ziemskie (m/s^2)
L = 1.0  # Długość wahadła (m)
theta0 = np.pi / 4  # Amplituda początkowa (rad)

# Funkcja podcałkowa h(theta)
def h_integrand(theta, k):
    return 1 / np.sqrt(1 - k**2 * np.sin(theta)**2)

# Funkcja obliczająca h(theta0)
def compute_h(theta0, N=50):
    k = np.sin(theta0 / 2)
    return simps(lambda theta: h_integrand(theta, k), 0, np.pi / 2, N)

# Przybliżenie harmoniczne
h_approx = np.pi / 2

# Obliczenia dla theta0 = 15, 30, 45 stopni
theta_values = [15, 30, 45]
results = {theta: compute_h(theta) for theta in theta_values}

# Wyświetlenie wyników i porównanie z przybliżeniem harmonicznym
print("Wyniki:")
for theta, h_val in results.items():
    print(f"h({theta}°) = {h_val:.6f}, różnica z h(0) = {abs(h_val - h_approx):.6f}")


Wyniki:
h(15°) = 2.492029, różnica z h(0) = 0.921232
h(30°) = 1.793725, różnica z h(0) = 0.222929
h(45°) = 1.678955, różnica z h(0) = 0.108159


# zad 5

In [13]:
# Funkcja do całkowania: f(x) = ln(x) / (x^2 - 2x + 2)
def integrand(x):
    return np.log(x) / (x**2 - 2*x + 2)

# Zakres całkowania: a=1, b=math.pi
a = 1
b = math.pi

for m in [2, 4]:
    result = gaussQuad(integrand, a, b, m)
    print(f"Całka dla {m} węzłów: {result:.6f}")

Całka dla 2 węzłów: 0.606725
Całka dla 4 węzłów: 0.584768


# zad 6

In [35]:
# Definicje funkcji i ich pochodnych
def f1(x): return x**3 - 2*x
def f1_prime(x): return 3*x**2 - 2

def f2(x): return np.sin(x)
def f2_prime(x): return np.cos(x)

def f3(x): return np.exp(x)
def f3_prime(x): return np.exp(x)

# Różnica centralna
def central_difference(f, x, h):
    return (f(x + h) - f(x - h)) / (2 * h)

# Punkty testowe
points = [
    {"func": f1, "deriv": f1_prime, "x": 1, "label": "f'_1(1) = 1"},
    {"func": f2, "deriv": f2_prime, "x": np.pi / 3, "label": "f'_2(π/3) = 1/2"},
    {"func": f3, "deriv": f3_prime, "x": 0, "label": "f'_3(0) = 1"}
]

# Kroki h
h_values = [0.1, 0.01, 0.001]

# Zbieranie wyników
rows = []

for point in points:
    # Obliczenie dla każdego h
    errors = []
    for h in h_values:
        approx = central_difference(point["func"], point["x"], h)
        error = point["deriv"](point["x"]) - approx
        errors.append(error)
    
    # Dodanie wiersza z obliczonymi błędami
    row = {
        "Pochodne": point["label"],
        "h": h_values[0],  # Zawsze h=0.1 w pierwszym wierszu
        "f ′(x) − Df": errors[0],
        "f ′(x) − Dc2": errors[1],
        "f ′(x) − Dc4": errors[2]
    }
    rows.append(row)
    
    # Powtarzanie dla h=0.01
    row_01 = row.copy()
    row_01["h"] = h_values[1]
    row_01["f ′(x) − Df"] = errors[1]
    rows.append(row_01)
    
    # Powtarzanie dla h=0.001
    row_001 = row.copy()
    row_001["h"] = h_values[2]
    row_001["f ′(x) − Df"] = errors[2]
    rows.append(row_001)

# Tworzenie tabeli z pandas
df = pd.DataFrame(rows)

# Wyświetlenie tabeli
df

,Pochodne,h,f ′(x) − Df,f ′(x) − Dc2,f ′(x) − Dc4
0,f'_1(1) = 1,0.100,-1.000000e-02,-0.000100,-1.000000e-06
1,f'_1(1) = 1,0.010,-1.000000e-04,-0.000100,-1.000000e-06
2,f'_1(1) = 1,0.001,-1.000000e-06,-0.000100,-1.000000e-06
3,f'_2(π/3) = 1/2,0.100,8.329168e-04,0.000008,8.333340e-08
4,f'_2(π/3) = 1/2,0.010,8.333292e-06,0.000008,8.333340e-08
5,f'_2(π/3) = 1/2,0.001,8.333340e-08,0.000008,8.333340e-08
6,f'_3(0) = 1,0.100,-1.667500e-03,-0.000017,-1.666667e-07
7,f'_3(0) = 1,0.010,-1.666675e-05,-0.000017,-1.666667e-07
8,f'_3(0) = 1,0.001,-1.666667e-07,-0.000017,-1.666667e-07


In [22]:
# Dane z tabeli
x_vals = np.array([0.0, 0.1, 0.2, 0.3, 0.4])
f_vals = np.array([0.000000, 0.078348, 0.138910, 0.192916, 0.244981])

# Funkcja do obliczania różnic centralnych
def central_difference_table(x_vals, f_vals, x, h):
    idx = np.where(np.isclose(x_vals, x))[0][0]
    f_x_plus_h = f_vals[idx + 1]
    f_x_minus_h = f_vals[idx - 1]
    return (f_x_plus_h - f_x_minus_h) / (2 * h)

# Obliczenia dla różnych kroków
h_values = [0.1, 0.2]
results = []

for h in h_values:
    f_prime = central_difference_table(x_vals, f_vals, 0.2, h)
    results.append({"h": h, "f'(0.2)": f_prime})

# Wyświetlenie wyników w tabeli
df = pd.DataFrame(results)
print(df)


     h  f'(0.2)
0  0.1  0.57284
1  0.2  0.28642
